<a href="https://colab.research.google.com/github/amithpaul/LSTMPdM/blob/main/LSTMtrain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# prompt: import numpy as np
# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import LSTM, Dense, Input
# from tensorflow.keras.callbacks import EarlyStopping
# install libraries for these

!pip install numpy tensorflow



In [7]:
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Dense, Input
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)

# Load preprocessed dataset
data = np.load("scaled_dataset.npy").astype(np.float32)  # Ensure float32
print("Shape of loaded data:", data.shape)
input_dim = data.shape[1]  # Number of features

Shape of loaded data: (22132, 49)


In [8]:
# Create sequences
seq_length = 100  # Adjust based on your dataset
def create_sequences(data, seq_length):
    sequences = []
    for i in range(len(data) - seq_length):
        sequences.append(data[i : i + seq_length])
    return np.array(sequences)

X_train = create_sequences(data, seq_length)

In [9]:
# LSTM Autoencoder
timesteps, features = X_train.shape[1], X_train.shape[2]

input_layer = Input(shape=(timesteps, features))
encoded = LSTM(64, activation="relu", return_sequences=True, recurrent_dropout=0.2)(input_layer)
encoded = LSTM(32, activation="relu", return_sequences=False, recurrent_dropout=0.2)(encoded)

decoded = Dense(64, activation="relu")(encoded)
decoded = Dense(features, activation="sigmoid")(decoded)

autoencoder = Model(input_layer, decoded)
autoencoder.compile(optimizer="adam", loss="mse",jit_compile=True)

In [5]:
# Train the model
autoencoder.fit(
    X_train, X_train[:, -1, :],
    epochs=100, batch_size=32,
    validation_split=0.1,
    callbacks=[early_stop]
)


Epoch 1/100
620/620 ━━━━━━━━━━━━━━━━━━━━ 38s 46ms/step - loss: 0.0510 - val_loss: 0.0652
Epoch 2/100
620/620 ━━━━━━━━━━━━━━━━━━━━ 28s 34ms/step - loss: 0.0091 - val_loss: 0.0280
Epoch 3/100
620/620 ━━━━━━━━━━━━━━━━━━━━ 21s 34ms/step - loss: 0.0057 - val_loss: 0.0263
Epoch 4/100
620/620 ━━━━━━━━━━━━━━━━━━━━ 41s 34ms/step - loss: 0.0048 - val_loss: 0.0237
Epoch 5/100
620/620 ━━━━━━━━━━━━━━━━━━━━ 41s 34ms/step - loss: 0.0035 - val_loss: 0.0139
Epoch 6/100
620/620 ━━━━━━━━━━━━━━━━━━━━ 40s 34ms/step - loss: 0.0027 - val_loss: 0.0172
Epoch 7/100
620/620 ━━━━━━━━━━━━━━━━━━━━ 41s 34ms/step - loss: 0.0023 - val_loss: 0.0157
Epoch 8/100
620/620 ━━━━━━━━━━━━━━━━━━━━ 41s 34ms/step - loss: 0.0022 - val_loss: 0.0154
Epoch 9/100
620/620 ━━━━━━━━━━━━━━━━━━━━ 41s 34ms/step - loss: 0.0020 - val_loss: 0.0147
Epoch 10/100
620/620 ━━━━━━━━━━━━━━━━━━━━ 41s 34ms/step - loss: 0.0019 - val_loss: 0.0152
Epoch 11/100
620/620 ━━━━━━━━━━━━━━━━━━━━ 41s 34ms/step - loss: 0.0018 - val_loss: 0.0155
Epoch 12/100
620/62

In [6]:
# Save the model
autoencoder.save("lstm_autoencoder.keras")